In [1]:
# Built-in modules
import math
import random

# Imports from Qiskit
from qiskit import QuantumCircuit
from qiskit.circuit.library import GroverOperator, MCMT, ZGate
from qiskit.visualization import plot_distribution

# Imports from Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler

import time

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue

from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    #token='2ce8efea0ea2f41cfe2db92f4771b22c101f583f98e4871b0893c8efb5c5cb8216928ae2ab16b108f59f79c36d5b711f5203a665cb65900e384b3a452c693f4b'
    #token='9e15651de6348bbb0bf0d12c502ab75780b008a847272ccd8bdcff8307ad4d4370c59ee4c8c4e3183ff91d6d9ab065acf06a58f82bec703275f33cd1d6dccaab'
    token="6bda2c672ab37cefe95614fcd46142eefa756ef230ebece16102cd421b140787256bfd4a1b58e49425f49472ef080536599d79d698d1d1ddb9ed2f1a33c705f9"
)


In [3]:
def grover_oracle(marked_states):
    """Build a Grover oracle for multiple marked states

    Here we assume all input marked states have the same number of bits

    Parameters:
        marked_states (str or list): Marked states of oracle

    Returns:
        QuantumCircuit: Quantum circuit representing Grover oracle
    """
    if not isinstance(marked_states, list):
        marked_states = [marked_states]
    # Compute the number of qubits in circuit
    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)
    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bit-string to match Qiskit bit-ordering
        rev_target = target[::-1]
        # Find the indices of all the '0' elements in bit-string
        zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]
        # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
        # where the target bit-string has a '0' entry
        qc.x(zero_inds)
        qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
        qc.x(zero_inds)
    return qc

In [4]:
def run_grover_experiment():
    num_qubits = 3

    random_marked_state = ''.join(random.choice('01') for _ in range(num_qubits))

    marked_states = [random_marked_state]

    oracle = grover_oracle(marked_states)
    
    backend = service.least_busy(operational=True, simulator=False)
    
    grover_op = GroverOperator(oracle)


    optimal_num_iterations = math.floor(
        math.pi / (4 * math.asin(math.sqrt(len(marked_states) / 2**grover_op.num_qubits)))
    )
    qc = QuantumCircuit(grover_op.num_qubits)
    qc.h(range(grover_op.num_qubits))
    qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
    qc.measure_all()
    target = backend.target
    pm = generate_preset_pass_manager(target=target, optimization_level=3)

    circuit_isa = pm.run(qc)
    sampler = Sampler(mode=backend)
    sampler.options.default_shots = 10_000
    result = sampler.run([circuit_isa]).result()
    
    dist = result[0].data.meas.get_counts()
    return random_marked_state, dist


In [5]:
import concurrent.futures

def run_all_experiments_concurrently(num_repeats, max_concurrent_tasks=3):
    results = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_concurrent_tasks) as executor:
        futures = {executor.submit(run_grover_experiment): i for i in range(num_repeats)}

        for future in concurrent.futures.as_completed(futures):
            try:
                marked_state, dist = future.result()
                results.append({
                    "Run": futures[future] + 1, 
                    "Marked State": marked_state,
                    "Result Distribution": dist
                })
                print(f"Experiment {futures[future] + 1} completed.")
            except Exception as e:
                print(f"Error in experiment {futures[future] + 1}: {e}")

    return results

num_repeats = 40
max_concurrent_tasks = 2
results = run_all_experiments_concurrently(num_repeats, max_concurrent_tasks)

Experiment 2 completed.
Experiment 1 completed.
Experiment 3 completed.
Error in experiment 5: 'One or more of the arguments are empty'
Experiment 4 completed.
Experiment 6 completed.
Experiment 8 completed.
Error in experiment 9: 'One or more of the arguments are empty'
Error in experiment 10: 'One or more of the arguments are empty'
Experiment 7 completed.
Experiment 11 completed.
Experiment 12 completed.
Experiment 13 completed.
Error in experiment 15: 'One or more of the arguments are empty'
Experiment 14 completed.
Experiment 16 completed.
Experiment 17 completed.
Experiment 18 completed.
Error in experiment 20: 'One or more of the arguments are empty'
Experiment 19 completed.
Experiment 21 completed.
Experiment 22 completed.
Experiment 23 completed.
Experiment 24 completed.
Experiment 25 completed.
Experiment 26 completed.
Error in experiment 28: 'One or more of the arguments are empty'
Error in experiment 29: 'One or more of the arguments are empty'
Experiment 27 completed.
Expe

D:\anaconda3\Lib\site-packages\qiskit_ibm_runtime\qiskit_runtime_service.py:849: UserWarning: Your current pending jobs are estimated to consume 19.928299008415262 quantum seconds, but you only have 16 quantum seconds left in your monthly quota; therefore, it is likely this job will be canceled
  warnings.warn(warning_message)


Experiment 38 completed.
Experiment 39 completed.
Error in experiment 40: 'Failed to run program: \'403 Client Error: Forbidden for url: https://api.quantum.ibm.com/runtime/jobs. {"errors":[{"message":"Job create exceeds open plan job usage limits","code":4317,"solution":"Please wait until the beginning of next month to submit more jobs when your quota will reset.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}\''
